In [68]:
from torchvision import models, transforms,datasets
from PIL import Image
import  torch
import pandas as pd
import torch.nn as nn
import os
import matplotlib.pyplot as plt
from collections import Counter
from src.model import SkinCancerModel
from src.dataset import MelanomaDataset,data_preprocessing,Normalize,MelanomaImageDataset
from torch.utils.data import DataLoader
from src import model_training
from torch.optim import Adam
import time
from sklearn.model_selection import train_test_split
torch.cuda.CUDA_LAUNCH_BLOCKING=1

In [69]:
from src.model import SkinCancerModel

In [70]:
model_extract_feature_from_images=SkinCancerModel(True,"efficientnet_b1")

In [71]:
model_extract_feature_from_images.classifier=nn.Sequential(nn.Dropout(0.25), nn.Linear(1280, 256), nn.ReLU(inplace=True), nn.Dropout(0.25),  nn.Linear(256, 128))

In [109]:
data_after=pd.read_csv("../dataset/preprocessed/after.csv")
data_after.head()

,image_id,dx,age,sex,localization_abdomen,localization_acral,localization_back,localization_chest,localization_ear,localization_face,localization_foot,localization_genital,localization_hand,localization_lower extremity,localization_neck,localization_scalp,localization_trunk,localization_upper extremity
0,ISIC_0027419,0,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,ISIC_0025030,0,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,ISIC_0026769,0,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,ISIC_0025661,0,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,ISIC_0031633,0,75.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [110]:
features=data_after.iloc[:,2:-1]
features.head()

,age,sex,localization_abdomen,localization_acral,localization_back,localization_chest,localization_ear,localization_face,localization_foot,localization_genital,localization_hand,localization_lower extremity,localization_neck,localization_scalp,localization_trunk
0,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,80.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,75.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [74]:
for i in range(len(data_after)):
        print(features.iloc[i,:])
        break

age                             80.0
sex                              1.0
localization_abdomen             0.0
localization_acral               0.0
localization_back                0.0
localization_chest               0.0
localization_ear                 0.0
localization_face                0.0
localization_foot                0.0
localization_genital             0.0
localization_hand                0.0
localization_lower extremity     0.0
localization_neck                0.0
localization_scalp               1.0
localization_trunk               0.0
Name: 0, dtype: float64


In [75]:
data=data_after.iloc[:,0:1]
data.head()


,image_id
0,ISIC_0027419
1,ISIC_0025030
2,ISIC_0026769
3,ISIC_0025661
4,ISIC_0031633


In [120]:
data_after.iloc[0,0]

'ISIC_0027419'

In [101]:
for i in range(len(data)):
        transform=transforms.PILToTensor()

        dataset_images_path="../dataset/preprocessed/ham1000"
        image_name=data_after.iloc[i,0]
        path=f"{dataset_images_path}/{image_name}.jpg"
        image= Image.open(path)
        image=transform(image)
        image=image.float()
        image=torch.unsqueeze(image,0)

        output_of_first_model=model_extract_feature_from_images(image)
        break
        ############################################################################################





In [112]:
features_tensor=torch.tensor(features.iloc[0].to_numpy()).unsqueeze(0)


In [107]:
output_of_first_model.shape

torch.Size([1, 128])

In [113]:
features_tensor.shape

torch.Size([1, 15])

In [105]:
features=features.unsqueeze(1)

In [115]:
input_into_second_model=torch.cat((output_of_first_model,features_tensor),dim=1)

In [116]:
input_into_second_model.shape

torch.Size([1, 143])

In [117]:
input_into_second_model=input_into_second_model.float()

In [90]:
input_into_second_model.size()

torch.Size([1, 143])

In [118]:
model = nn.Sequential(nn.Linear(143, 71), nn.ReLU(),nn.Linear(71, 20), nn.ReLU(), nn.Linear(20, 1),nn.Sigmoid())

In [119]:
model(input_into_second_model)

tensor([[0.5241]], grad_fn=<SigmoidBackward0>)